## 파이썬 프로파일러
`cProfile`과 `profile`은 파이썬 프로그램이 얼마나 자주, 그리고 오랫동안 실행되었는지 기술하는 통계입니다. 통계는 pstats 모듈을 통해 보고서로 포매팅 가능합니다.

파이썬 표준 라이브러리는 같은 프로파일링 인터페이스로 구현된 두 가지를 제공하는데, `cProfile`과 `profile`입니다.

> [파이썬 프로파일러](https://docs.python.org/3/library/profile.html)

### `cProfile`
파이썬에서는 상당한 오버헤드가 발생하는 장시간 실행되는 프로그램을 프로파일링을 할 경우에 `cProfile`을 주로 추천합니다. 

이와 반대로 `profile`은 순수 파이썬으로 구현되어 있어 프로파일링에 대한 기능을 확장하는데 용이하겠지만, 상당한 오버헤드가 발생하는 코드에 대해서는 효율적이지 않습니다.

#### usage
cProfile을 실행하면, 아래와 같이 함수에 대해서 프로파일하며, 결과를 기술합니다.

In [ ]:
import cProfile
import re
cProfile.run('re.compile("foo|bar")')

총 214개의 함수가 호출되었으며 이 중 207개의 primitive 호출이 생겼는데, 즉 214개에서 207개를 제외한 나머지 7개의 함수가 재귀적으로 호출되었다는 의미입니다.

각 컬럼의 의미는 아래와 같습니다.

- ncalls : 호출 수
- tottime : 함수별 런타임
- percall : tottime을 ncalls로 나눈 값
- cumtime : 함수와 서브함수에서 사용된 누적시간(호출 -> 종료), * 이 값은 재귀함수에서도 마찬가지로 적용됩니다.
- percall : cumtime을 primitive 호출을 나눈 값
- filename:lineno(function) : 파일명:줄번호(함수명)

이때 ncalls에 3/1 과 같은 함수는 재귀 함수로 호출된 것이라 이해할 수 있습니다.

아래는 임의로 생성한 함수입니다. 아래의 함수를 프로파일링 해보겠습니다.

In [23]:
import pstats
a = list(range(1, 1000000))

def func1(a):
    for i in a:
        if i == 999999:
            print(999999)

def func2(a):
    b = []
    for i in a:
        b.append(i * 2)
    return b

def main():
    func1(a)
    func2(a)

cProfile.run('main()')

999999
         1000040 function calls in 0.451 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.020    0.020    0.451    0.451 <ipython-input-23-0fd956a41d69>:15(main)
        1    0.076    0.076    0.077    0.077 <ipython-input-23-0fd956a41d69>:4(func1)
        1    0.245    0.245    0.354    0.354 <ipython-input-23-0fd956a41d69>:9(func2)
        1    0.000    0.000    0.451    0.451 <string>:1(<module>)
        3    0.000    0.000    0.001    0.000 iostream.py:195(schedule)
        2    0.000    0.000    0.000    0.000 iostream.py:308(_is_master_process)
        2    0.000    0.000    0.000    0.000 iostream.py:321(_schedule_flush)
        2    0.000    0.000    0.001    0.000 iostream.py:384(write)
        3    0.000    0.000    0.000    0.000 iostream.py:91(_event_pipe)
        3    0.001    0.000    0.001    0.000 socket.py:438(send)
        3    0.000    0.000    0.000    0.000 threading.py:1050(_wait_f

### 프로파일링을 통한 최적화

- 처리속도 측정 이후 우선순위를 두고 개선
- 반복문을 제거할 수 있는지 검토(zip, map, filter 등)
- 내장함수나 라이브러리를 활용
- 지역변수를 사용(global 권장하지 않음)
- 가비지 컬렉션이 있으나, 사용이 끝난 규모가 큰 변수에 대해서는 `del` 삭제
- numpy, pandas 에서 inplace로 직접 수정하거나, 사본을 생성하지 않는 것이 메모리 효율에 좋음